In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# default_exp optionsseasonal

# optionsseasonal

> seasonal equity performance is used to guestimate performance probability

In [ ]:
#hide
from nbdev.showdoc import *
from hesap.pricedata import get_history_yf, get_history

In [ ]:
#export

"""This module contains methods to create and update daily price and weekly
performance for the desired assets locally. Also for a given week, the most
optimal assets (with highest performance probability based on the history)
are identified for possible MattChoi's type option strategy.
by faf
Copyright 2021 Fafs
License: http://creativecommons.org/licenses/by/4.0/
"""

import os

import numpy as np
import pandas as pd

from typing import List


def weekly_returns(df):
    """
    Processes the daily adjusted close prices to produce 35 to 45 week forward
    performances
    """
    df.loc[:, 'year'] = df.index.isocalendar().year
    df.loc[:, 'week'] = pd.Int64Index(df.index.isocalendar().week)
    for n in range(35, 46):
        df.loc[:, f'forward{n}'] = df.loc[:, 'adj_close'].shift(-n) / df.loc[:, 'adj_close']

    rets, forwards, means = {}, {}, []
    for week in range(2, 52):
        if week not in df.week.unique(): continue
        forwards[week] = df.loc[df.week == week,
                                ['forward35', 'forward36', 'forward37',
                                 'forward38', 'forward39', 'forward40',
                                 'forward41', 'forward42', 'forward43',
                                 'forward44', 'forward45', 'year']]

        years = sorted(forwards[week].year.unique())
        rets[week] = []
        for year in years:
            returns = []
            for n in range(35,46):
                returns.extend(forwards[week].loc[(forwards[week].year == year), f'forward{n}'].tolist())
            rets[week].append({
                'year': year,
                'min': min(returns),
                'max': max(returns),
                'mean': np.mean(returns),
                'std': np.std(returns),
                'up_conf': np.sum([1 for r in returns if r > 1]) / len(returns) * 100,
                'dn_conf': np.sum([1 for r in returns if r < 1]) / len(returns) * 100
            })
        rets[week] = pd.DataFrame(rets[week])

        up_mean = round(rets[week].up_conf.mean(), 2)
        dn_mean = round(rets[week].dn_conf.mean(), 2)
        means.append({'week': week, 'up_mean': up_mean, 'dn_mean': dn_mean})
        if up_mean > 80 or dn_mean > 80:
            print(f'{week:2}:\t{up_mean}\t{dn_mean}')
    return rets, means


def build_weekly_db(tickers: List[str],
                    db_path: str,
                    start_date: str = '2000-01-01'
                   ) -> None:
    if tickers is None or len(tickers) == 0: return
    for d in [db_path, f'{db_path}/prices', f'{db_path}/rets',
              f'{db_path}/mean_rets']:
        os.makedirs(d, exist_ok=True)
    for i, ticker in enumerate(tickers):
        prices = get_history_yf(ticker.replace('.', '-'),
                                start_date=start_date)
        prices.to_csv(f'{db_path}/prices/{ticker}.csv')
        print(f'={i+1:4}{"="*4}{ticker:4}{"="*20}')
        rets, means = weekly_returns(prices)

        try:
            pd.DataFrame.from_dict(data=rets, orient='index').to_csv(f'{db_path}/rets/{ticker}.csv', header=False)
            pd.DataFrame(means).to_csv(f'{db_path}/mean_rets/{ticker}.csv')
        except:
            print(f'could not create dataframe for {ticker}')

In [ ]:
#hide
db_path = '/Users/fahrisurucu/Projects/my_data/'
ticker_path = '/Users/fahrisurucu/Projects/my_secrets/sp500_20210613.tsv'
tickers = pd.read_csv(ticker_path, sep='\t').loc[:, 'ticker'].to_list()
build_weekly_db(tickers[480:], db_path, '1990-01-01')

[*********************100%***********************]  1 of 1 completed
=   1====VNT ====================
 2:	0.0	100.0
 3:	0.0	100.0
 4:	0.0	100.0
 5:	0.0	100.0
 6:	0.0	100.0
 7:	4.55	95.45
10:	98.18	1.82
11:	100.0	0.0
12:	100.0	0.0
13:	100.0	0.0
14:	98.18	0.0
39:	9.09	90.91
42:	100.0	0.0
43:	100.0	0.0
44:	100.0	0.0
45:	100.0	0.0
46:	100.0	0.0
47:	100.0	0.0
50:	98.18	1.82
could not create dataframe for VNT
[*********************100%***********************]  1 of 1 completed
=   2====VNO ====================


/Users/fahrisurucu/opt/anaconda3/envs/fsx_env/lib/python3.7/site-packages/pandas/core/internals/construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


[*********************100%***********************]  1 of 1 completed
=   3====VMC ====================


/Users/fahrisurucu/opt/anaconda3/envs/fsx_env/lib/python3.7/site-packages/pandas/core/internals/construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


[*********************100%***********************]  1 of 1 completed
=   4====WRB ====================


...

/Users/fahrisurucu/opt/anaconda3/envs/fsx_env/lib/python3.7/site-packages/pandas/core/internals/construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


[*********************100%***********************]  1 of 1 completed
=  30====ZION====================


/Users/fahrisurucu/opt/anaconda3/envs/fsx_env/lib/python3.7/site-packages/pandas/core/internals/construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


[*********************100%***********************]  1 of 1 completed
=  31====ZTS ====================
12:	84.24	15.76
13:	94.65	5.35
14:	93.94	5.86
15:	80.4	15.56
29:	82.73	17.27
30:	87.27	12.73
31:	83.41	16.36
44:	84.32	15.68


/Users/fahrisurucu/opt/anaconda3/envs/fsx_env/lib/python3.7/site-packages/pandas/core/internals/construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [ ]:
prices = get_history_yf('VNT')
prices.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,adj_close,Volume
Date,,,,,,
2020-09-24,36.00,36.0,34.00,34.0,33.975784,743500
2020-09-28,35.00,35.0,33.00,33.0,32.976494,150500
2020-09-29,30.50,30.5,30.50,30.5,30.478275,220200
2020-09-30,30.75,31.0,30.75,31.0,30.977919,21900
2020-10-01,34.00,34.0,34.00,34.0,33.975784,43600


In [ ]:
rets, means = weekly_returns(prices)

 2:	0.0	100.0
 3:	0.0	100.0
 4:	0.0	100.0
 5:	0.0	100.0
 6:	0.0	100.0
 7:	4.55	95.45
10:	98.18	1.82
11:	100.0	0.0
12:	100.0	0.0
13:	100.0	0.0
14:	98.18	0.0


AttributeError: 'DataFrame' object has no attribute 'up_conf'

In [ ]:
df = prices
if True:
    """
    Processes the daily adjusted close prices to produce 35 to 45 week forward
    performances
    """
    df.loc[:, 'year'] = df.index.isocalendar().year
    df.loc[:, 'week'] = pd.Int64Index(df.index.isocalendar().week)
    for n in range(35, 46):
        df.loc[:, f'forward{n}'] = df.loc[:, 'adj_close'].shift(-n) / df.loc[:, 'adj_close']

    rets, forwards, means = {}, {}, []
    for week in range(2, 52):
        print(week)
        if week not in df.week.unique(): continue
        forwards[week] = df.loc[df.week == week,
                                ['forward35', 'forward36', 'forward37',
                                 'forward38', 'forward39', 'forward40',
                                 'forward41', 'forward42', 'forward43',
                                 'forward44', 'forward45', 'year']]

        years = sorted(forwards[week].year.unique())
        rets[week] = []
        for year in years:
            returns = []
            for n in range(35,46):
                returns.extend(forwards[week].loc[(forwards[week].year == year), f'forward{n}'].tolist())
            rets[week].append({
                'year': year,
                'min': min(returns),
                'max': max(returns),
                'mean': np.mean(returns),
                'std': np.std(returns),
                'up_conf': np.sum([1 for r in returns if r > 1]) / len(returns) * 100,
                'dn_conf': np.sum([1 for r in returns if r < 1]) / len(returns) * 100
            })
        rets[week] = pd.DataFrame(rets[week])

        up_mean = round(rets[week].up_conf.mean(), 2)
        dn_mean = round(rets[week].dn_conf.mean(), 2)
        means.append({'week': week, 'up_mean': up_mean, 'dn_mean': dn_mean})
        if up_mean > 80 or dn_mean > 80:
            print(f'{week:2}:\t{up_mean}\t{dn_mean}')

2
 2:	0.0	100.0
3
 3:	0.0	100.0
4
 4:	0.0	100.0
5
 5:	0.0	100.0
6
 6:	0.0	100.0
7
 7:	4.55	95.45
8
9
10
10:	98.18	1.82
11
11:	100.0	0.0
12
12:	100.0	0.0
13
13:	100.0	0.0
14
14:	98.18	0.0
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
39:	9.09	90.91
40
41
42
42:	100.0	0.0
43
43:	100.0	0.0
44
44:	100.0	0.0
45
45:	100.0	0.0
46
46:	100.0	0.0
47
47:	100.0	0.0
48
49
50
50:	98.18	1.82
51


In [ ]:
rets.keys()

dict_keys([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])

In [ ]:
ticker = 'AAPL'
pd.DataFrame.from_dict(rets, orient='index').to_csv(f'{db_path}/rets/{ticker}.csv', header=False)

/Users/fahrisurucu/opt/anaconda3/envs/fsx_env/lib/python3.7/site-packages/pandas/core/internals/construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])
